In [ ]:

import torch as t
import torch.nn as nn 
import torch.nn.functional as F 
from torch import Tensor
from jaxtyping import Int, Float
from dataclasses import dataclass
import einops

In [ ]:
class StridedAttention(nn.Module):
    def __init__(self, num_heads=2, d_model=3, d_head=2, horizontal_stride=2):
        super().__init__()
        self.num_heads = num_heads
        self.horizontal_stride = horizontal_stride
        self.W_Q = nn.Parameter(t.randn(d_model, d_head))
        self.W_K = nn.Parameter(t.randn(d_model, d_head))
        self.W_V = nn.Parameter(t.randn(d_model, d_head))

    def forward(self, embed: Float[Tensor, "b s d_m"]) -> Float[Tensor, "b s d_m"]:
        b, s, d_m = embed.shape
        Q = embed @ self.W_Q # b s_q d_head
        K = embed @ self.W_K # b s_k d_head
        V = embed @ self.W_V # b s_v d_head

        # 2 heads
        # b 1 s_q d_head
        Q = Q[:, None, ...]
        # TODO: Think of a non memory moving way to do this.
        # b 2 s_q d_head
        Q = Q.repeat(1,2,1,1)

        # b 1 s_k d_head
        K = K[:, None, ...]
        V = V[:, None, ...]
        # stride_width = seq_len // num_heads
        print(V.shape)

        # horizontal stride shape b 1 s_k//stride d_head
        K = K[:, :, range(0, s, self.horizontal_stride), :]
        print(K.shape)

        # range 1..... seq_Q
        # seq_Q, s_k//stride

        

        # K_t = K.transpose(1, 2) # b d_head s_k
        # Q_K_t = Q @ K_t # GOAL: b s_q s_k//stride
        
        # sf(qk/scale)v


B=4
S=128
D=256
input = t.randn((B, S, D))
print(input.shape)
attn = StridedAttention(2, D, D//2)        
attn(input)


torch.Size([4, 128, 256])
torch.Size([4, 1, 128, 128])


In [29]:
test = t.arange(0, 3)[:, None] + t.arange(0, 24, 8)[None, :]
print(test.shape)
print(test)


torch.Size([3, 3])
tensor([[ 0,  8, 16],
        [ 1,  9, 17],
        [ 2, 10, 18]])


In [ ]:
toks = t.tensor([
    [1,2,3,4,5,6],
    [7,8,9,10,11,12],
])

In [ ]:
seq_len, d_model = toks.shape
num_heads = 2
head_dim = d_model // num_heads

viewed = toks.view(num_heads, seq_len, head_dim)

In [8]:
viewed[1, :, :]

tensor([[ 7,  8,  9],
        [10, 11, 12]])